In [29]:
import torch
import torchvision.transforms as transforms
from LookGenerator.datasets.tps_dataset import ShirtsDataset
from torch.utils.data import DataLoader
from LookGenerator.networks.bpgm.model.models import BPGM
from LookGenerator.networks.bpgm.train_bpgm import train_bpgm
import torch.nn.functional as F
import numpy as np
import os
import PIL.Image as Image
import matplotlib.pyplot as plt
from LookGenerator.networks_training.utils import check_path_and_creat
from LookGenerator.networks.utils import save_model, load_model

In [2]:
transform = transforms.Compose([
    transforms.Resize((256,192)),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])
mask_transform = transforms.Compose([
    transforms.Resize((256,192)),
])


In [3]:
batch_size = 64
num_workers = 12
pin_memory = True

In [4]:
dataset = ShirtsDataset(
    root=r"C:\Users\DenisovDmitrii\Desktop\zalando-hd-resize\train",
    transform=transform,
    transform_mask=mask_transform
)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=pin_memory, num_workers=num_workers)

In [5]:
save_directory = r'C:\Users\DenisovDmitrii\OneDrive - ITMO UNIVERSITY\peopleDetector\tps\weights\test'
check_path_and_creat(save_directory)

True

In [6]:
for data in dataloader:
    break

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [8]:
model = BPGM(device=device)

initialization method [normal]
initialization method [normal]


In [18]:
loss = train_bpgm(dataloader=dataloader,
                  model=model, device=device, epochs=1)

100%|██████████| 182/182 [07:27<00:00,  2.46s/it]

[]


In [19]:
save_model(model.to('cpu'), path=f"{save_directory}\\epoch_01.pt")

# тест

In [24]:
test_dataset = ShirtsDataset(
    root=r"C:\Users\DenisovDmitrii\Desktop\zalando-hd-resize\test",
    transform=transform,
    transform_mask=mask_transform
)

In [25]:
topil = transforms.ToPILImage()

In [26]:
testdataloader = DataLoader(test_dataset, batch_size=1, pin_memory=True)

In [30]:
weights_dir = r"C:\Users\DenisovDmitrii\OneDrive - ITMO UNIVERSITY\peopleDetector\tps\weights\test\epoch_00.pt"

In [31]:
model = load_model(model, weights_dir)

In [36]:
count = 0
for data in testdataloader:
    mask = data['segmentation'].to(device)
    shirt = data['shirt'].to(device)
    shirt_mask = data['shirt_mask'].to(device)

    model = model.to(device)
    theta = model(mask, shirt)

    warped = F.grid_sample(shirt, theta, padding_mode='border', align_corners=True).to('cpu')

    warped_mask = F.grid_sample(shirt_mask, theta, padding_mode='border', align_corners=True).to('cpu')
    warped_mask = topil(warped_mask[0])
    warped_mask.show()
    warped = topil(warped[0]/2+0.5)
    warped.show()
    count += 1
    if count == 4:
        break

подготовка

In [ ]:
cloud_people = os.listdir("путь")

In [ ]:
# ИСПОЛЬЗОВАТЬ batch_size=1
for index, data in enumerate(dataloader):

    cloud_person = Image.open( "путь" + cloud_people[index])
    cloud_person = np.array(cloud_person)

    mask = data['segmentation'].to(device)
    shirt = data['shirt'].to(device)
    shirt_mask = data['shirt_mask'].to(device)

    model = model.to(device)
    theta = model(mask, shirt)

    warped = F.grid_sample(shirt, theta, padding_mode='border', align_corners=True).to('cpu')

    warped_mask = F.grid_sample(shirt_mask, theta, padding_mode='border', align_corners=True).to('cpu')
    warped_mask = np.array(topil(warped_mask[0]))
    warped = np.array(topil(warped[0]/2+0.5))

    wc = warped * warped_mask

    person = cloud_person + wc

    Image.fromarray(person, 'RGB').save("путь" + cloud_people[index])

    count += 1
    if count == 4:
        break